In [ ]:
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import shutil
import cv2

In [ ]:
def random_shape():
    x1, y1 = np.random.uniform(high=0.6, size=2)
    x2, y2 = np.random.uniform(low=0.3, size=2) * (1 - x1, 1 - y1) + (x1, y1)
    bounded_normal = np.clip(np.random.normal(loc=3, size=4) / 6, min=0, max=1)
    length = bounded_normal * np.array([x2 - x1, y2 - y1, x2 - x1, y2 - y1])
    points = np.array([
        [x1 + length[0], y1],
        [x2, y1 + length[1]],
        [x1 + length[2], y2],
        [x1, y1 + length[3]]
    ])
    return points

def lines(points, w, h):
    return (points * (w, h)).astype(int)

def random_colour():
    return np.random.randint(low=0, high=32, size=3).tolist()

def generate(n, backgrounds_path, images_path, labels_path):
    backgrounds = list(backgrounds_path.glob("*.jpg"))
    for i in range(1, n + 1):
        image = cv2.imread(np.random.choice(backgrounds))
        h, w, _ = image.shape
        shape = random_shape()
        image = cv2.Canny(image, 100, 200)
        image = cv2.polylines(image, [lines(shape, w, h)], True, (255, 255, 255), 2)
        cv2.imwrite(images_path / f"{i}.jpeg", image)
        with open(labels_path / f"{i}.txt", "w") as f:
            strings = map(str, np.ravel(shape).tolist())
            f.write(f"0 {' '.join(strings)}\n")

In [ ]:
dataset_path = Path("datasets/random")
backgrounds_path = Path("datasets/coco2017/images/val")

if dataset_path.exists():
    shutil.rmtree(dataset_path)
for n, path in [(1e4, "train"), (1e3, "val")]:
    images_path = dataset_path / "images" / path
    labels_path = dataset_path / "labels" / path
    images_path.mkdir(parents=True)
    labels_path.mkdir(parents=True)
    generate(int(n), backgrounds_path, images_path, labels_path)